In [33]:
import networkx as nx
from scipy.optimize import linprog
from sympy import Matrix, pprint, floor, Point, zeros
from linkages import *

def graph_to_matrix(G, pins={}):
    if type(pins) is Point: pins = {pins}
    M = Matrix()
    for edge in G.edges.data():
        edge_from, edge_to, edge_att = edge
        row = []
        t = -1 if edge_att[TYPE] == "strut" else 1
        for vertex in G.nodes:
            if vertex == edge_from and edge_from not in pins:
                row.extend([t*(edge_from.x-edge_to.x),t*(edge_from.y-edge_to.y)])
            elif vertex == edge_to and edge_to not in pins:
                row.extend([t*(edge_to.x-edge_from.x),t*(edge_to.y-edge_from.y)])
            else:
                row.extend([0,0])
        M = Matrix([M,row])
        if edge_att[TYPE] == "bar": M = Matrix([M,M.row(-1)*-1])
    return M

def change_edge(G,edge,val):
    a,b = edge
    graph[list(G.nodes)[a]][list(G.nodes)[b]][TYPE] = val
    
def check_rigidity(A): 
    b = zeros(A.rows,1)
    for i in range(0,A.cols):
        if A.col(i) == zeros(A.rows,1): continue
        c = zeros(1,A.cols)
        c[i] = -1
        res = linprog(c, A_ub=A,b_ub=b, method="simplex")
        if res['status'] == 0:
            for value in res['x']:
                if value > 0:
                    return "infinitesimal motion of:", value
        elif res['status'] == 3:
            return "inequalities are unbound--motion possible"
    return "the tensegrity is rigid"
    
graph = parallel_4_bar(True)
nx.set_edge_attributes(graph,"cable",TYPE)

change_edge(graph,(0,2),"strut")
change_edge(graph,(1,3),"strut")

A = graph_to_matrix(graph, {Point(0,0),Point(5,5)})
pprint(A)

print (check_rigidity(A))


⎡0  0  5   0   0  0  0   0 ⎤
⎢                          ⎥
⎢0  0  0   0   0  0  0   5 ⎥
⎢                          ⎥
⎢0  0  0   0   0  0  0   0 ⎥
⎢                          ⎥
⎢0  0  0   -5  0  0  0   0 ⎥
⎢                          ⎥
⎢0  0  -5  5   0  0  5   -5⎥
⎢                          ⎥
⎣0  0  0   0   0  0  -5  0 ⎦
the tensegrity is rigid
